In [2]:
import pymysql
import pandas as pd

In [1]:
import pymysql
import pandas as pd

# DataFrame creation
df = pd.DataFrame({
    'sepal_length': [5.1, 4.9, 4.7],
    'sepal_width': [3.5, 3.0, 3.2],
    'petal_length': [1.4, 1.4, 1.3],
    'petal_width': [0.2, 0.2, 0.2],
    'petal_width': [0.2, 0.2, 0.2],
    'petal_length': [1.4, 1.4, 1.3],
    'species': ['setosa', 'setosa', 'setosa']
})

try:
    # Connect
    conn = pymysql.connect(
        host='localhost',
        user='birat-gautam',
        password='NewStrongPasswordHere',
        database='scrapped_data_database_1'
    )
    with conn.cursor() as cursor:
        # Create table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Iris (
                sepal_length FLOAT,
                sepal_width FLOAT,
                petal_length FLOAT,
                petal_width FLOAT,
                species VARCHAR(20)
            )
        """)
        
        # Insert rows in bulk (more efficient)
        insert_query = """
            INSERT INTO Iris (sepal_length, sepal_width, petal_length, petal_width, species)
            VALUES (%s, %s, %s, %s, %s)
        """
        cursor.executemany(insert_query, df.values.tolist())
    
    # Commit
    conn.commit()

except Exception as e:
    print("Error:", e)

finally:
    if conn:
        conn.close()


In [4]:
# Retrieve the data back to verify
import pymysql
import pandas as pd

# Connect to MariaDB
conn = pymysql.connect(
    host='localhost',
    user='birat-gautam',
    password='NewStrongPasswordHere',
    database='scrapped_data_database_1'
)

# Use pandas to read the SQL table into a DataFrame
df_retrieved = pd.read_sql("SELECT * FROM Iris", conn)

conn.close()

# Show the DataFrame
df_retrieved.head()


/tmp/ipykernel_715616/2475506504.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_retrieved = pd.read_sql("SELECT * FROM Iris", conn)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
